In [3]:
#!pip install faker

In [37]:
#!pip install google-auth google-auth-oauthlib google-auth-httplib2


In [106]:
from google.oauth2 import service_account
import google.auth
import requests
import pandas as pd
import json
import urllib3
urllib3.disable_warnings()

pd.set_option('display.max_colwidth', None)


In [107]:
def busqueda_libros(parametro_busqueda):
    url = 'https://www.googleapis.com/books/v1/volumes?q='
    response = requests.get(url+parametro_busqueda, verify = False)

    r = response.content
    data = json.loads(r)

    df = pd.json_normalize(data, record_path =['items'])
    df.columns = [x.split('.')[1] if '.' in str(x) else x for x in df.columns]

    # Agrego columna con termino de busqueda para tener todo clar
    df['palabras_de_busqueda'] = busqueda_query
    return df

In [125]:
lista_palabras_busqueda = ['matematica','calculo','algebra','pensamiento computacional','integrales','determinantes','historia uruguay']
lista_dfs = []
columnas_deseadas = ['title','subtitle','publishedDate','pageCount','categories','language','webReaderLink','palabras_de_busqueda']

for frase in lista_palabras_busqueda:
    print('Buscando...')
    df_aux = busqueda_libros(frase)
    
    for i in columnas_deseadas:
        if i not in df_aux.columns:
            df_aux[f'{i}'] = None
    
    df_aux = df_aux[columnas_deseadas]
    
    lista_dfs.append(df_aux)
    print('Busqueda:', f'{frase}', '- completa')
    print('-------')   

Buscando...
Busqueda: matematica - completa
-------
Buscando...
Busqueda: algebra - completa
-------
Buscando...
Busqueda: pensamiento computacional - completa
-------
Buscando...
Busqueda: integrales - completa
-------
Buscando...
Busqueda: determinantes - completa
-------
Buscando...
Busqueda: historia uruguay - completa
-------


In [129]:
final_df = pd.concat(lista_dfs, ignore_index=True)

In [132]:
# guardo la base

final_df.to_csv('bases/libros.csv',sep='|')